Formats the feature files in order to get the feature file required by GPNN.

# Features required by GPNN
dict_keys(['metadata', 'num_obj', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation', 'i3d_feature_map', 'bboxes', 'cnn_bbox_feature'])

Put all the features of all the files in one pickle file and then write additional code in the dataloader to separate the features according to test, train and val splits.

In [5]:
import pickle as pkl

train_file = '/workspace/data/data_folder/o2o/all_features/gpnn/val.pkl'
d = pkl.load(open(train_file, 'rb'))

In [6]:
import numpy as np
import torchvision
import torch
from copy import deepcopy as copy

def roi_align(feature_map, boxes, num_obj, im_width, im_height):
    """
    Generates roi aligned feature vectors

    Args:
        feature_map (tensor([N, C, H, W]))  : 
        boxes (tensor([N_max, N, 4]))       : 
        num_obj (int)                       : 
        im_width (int)                      : 
        im_height (int)                     :

    Returns:
        _type_: _description_
    """
    # scaling the bounding boxes to lie within the feature map width and height    
    fmap_height, fmap_width = feature_map.shape[2:]

    boxes[:,:,0] *= (fmap_width/(im_width*1.0))
    boxes[:,:,2] *= (fmap_width/(im_width*1.0))

    boxes[:,:,1] *= (fmap_height/(im_height*1.0))
    boxes[:,:,3] *= (fmap_height/(im_height*1.0))
    
    # Copying feature maps for safety of the data
    feature_map = copy(feature_map.to(torch.device('cpu')))

    # boxes of shape torch.Size([max_num_obj, num_frame, 4])
    boxes_list = list(boxes.split(1, dim=1))
    boxes_list = [boxes_list[i].squeeze(1)[:num_obj,:] for i in range(len(boxes_list))]    

    pooler = torchvision.ops.RoIAlign(
                                        output_size=(1, 1),
                                        spatial_scale = 1.0,
                                        sampling_ratio=1
                                      )
    
    pooled_output = pooler(feature_map, boxes_list).squeeze()
    
    max_num_obj = boxes.shape[0]
    num_frame = boxes.shape[1]
    fmap_dim = feature_map.shape[1]
    
    resulting_tensor = torch.zeros(( num_frame, max_num_obj, fmap_dim))
    
    for frame_index in range(num_frame):
        
        ind0 = frame_index * num_obj
        ind1 = (frame_index + 1) * num_obj
        
        resulting_tensor[frame_index, :num_obj] = pooled_output[ind0:ind1]


    return resulting_tensor


# Master Feature Generator for VSGNet
import torch
import os
from copy import deepcopy as copy

def master_feature_generator( full_feature_filename ):

    # Get the feature from the vsgnet folder
    # Load the resnet 152 feature map
    # remove unnecessary keys
    # perform RoIalign on the resnet152 feature map
    # return the result
    
    # ['legend', 'metadata', 'num_obj', 'bboxes', 'lr', 'mr', 'cr', 'object_pairs',
    # 'num_relation', 'geometric_feature', '2d_cnn_feature_map', 'object_2d_cnn_feature', 
    # 'iou', 'distance', 'relative_spatial_feature', 'i3d_feature_map', 'object_i3d_feature', 'motion_feature']
    
    # required_keys = ['metadata', 'num_obj', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation', 'bboxes'] # 'i3d_feature_vec'

    required_keys =  ['metadata', 'num_obj', 'bboxes', 'lr', 'mr', 'cr', 'object_pairs', 'num_relation', 
                      'geometric_feature', 'object_2d_cnn_feature', 'iou', 'distance', 
                      'relative_spatial_feature', 'object_i3d_feature', 'motion_feature']

    full_feature = torch.load(full_feature_filename)

    new_dict = {}
    for r in required_keys:
        new_dict[r] = copy(full_feature[r])

    return new_dict

sample_pth = '/workspace/data/data_folder/o2o/gifs_11_features_ral_v2/_9DAGF1cJK8_1986_5.pt'
test_dict = master_feature_generator(sample_pth)

In [ ]:
from glob import glob

full_features_folder = '/workspace/data/data_folder/o2o/gifs_11_features_ral_v2'
all_feature_files = glob( os.path.join(full_features_folder, '*.pt') )

formatted_features = []
from tqdm import tqdm as tqdm

for f in tqdm(all_feature_files):
    formatted_dict = master_feature_generator(f)
    formatted_features.append(formatted_dict)

import pickle as pkl
path = '/workspace/data/data_folder/o2o/ral_formatted_features/gpnn/gpnn_feats.pkl'
f = open(path, 'wb')
pkl.dump(formatted_features, f)
f.close()